# Exercices pour l'évaluation des dérivés sur un arbre binomial FINA60211(A)

For the English version of this file, [click here](bino_exercises.ipynb).

Ce carnet de notes interactif et le module python associé `pedagogical_binomial_model` vous fournissent des outils pour générer vos propres exercices sur l'évaluation des options sur un arbre binomial. Vous pouvez utiliser ce carnet de notes pour :

- apprendre à construire des arbres binomiaux *forward*
- apprendre à calculer les probabilités d'états sur un arbre binomial
- apprendre à tarifier des options par la méthode récursive d'espérance risque-neutre et d'actualisation sur une seule période.
- apprendre à tarifier des options européennes en calculant l'espérance risque-neutre du valeur final.

Le module `pedagogical_binomial_model` suit les conventions et la notation de nos diapositives de cours.

## Travailler avec le module

Afin d'apprendre à utiliser le module, vous pouvez simplement lire la documentation dans le fichier `pedagogical_binomial_model.py` ou essayer le code `help` suivant :

In [17]:
import pedagogical_binomial_model as pbm
help(pbm.binomial_tree)

Help on class binomial_tree in module pedagogical_binomial_model:

class binomial_tree(builtins.object)
 |  binomial_tree(Stock_0, N_steps, h_time_step_length, r_int_rate, d_div_rate, s_volatility)
 |  
 |  Forward Binomial Tree for pricing Equity Derivatives.
 |  
 |  All math and notation follows the course slides for FINA60211(A) Princpes d'evaluation des produits derives
 |  
 |  Attributes
 |  ----------
 |  Stock_0 : float
 |      initial stock price
 |  N_steps : int
 |      number of steps on the tree model
 |  h_time_step_length : float
 |      length of a single time step (in years) on the tree: the tree describes stock price behavior over the period of length N_steps * h_time_step_length
 |  r_int_rate : float
 |      risk-free interest rate (NOT in percentage pts, i.e., for a rate of 4.2% input 0.042)
 |  d_div_rate : float
 |      dividend yield, (NOT in percentage pts, i.e., for a yield of 4.2% input 0.042)
 |  s_volatility : float
 |      annualized volatility of the sto

## Exemples

### Créer un arbre binomial pour le prix de l'action

Créez un arbre pour décrire l'évolution du prix de l'action :

- pendant trois mois
- par étapes d'un mois

Utilisez les paramètres suivants pour l'arbre :

- $S_0$ = 100
- $r_f$ = 2%
- $\delta$ = 1%
- $\sigma$ = 20%

In [18]:
stock_price = 100
number_of_model_steps = 3 # three months
length_of_model_step = 1/12 # a month is 1/12 years
risk_free_rate = 0.02
dividend_yield = 0.01
stock_volatility = 0.2 # annualized

stock_price_tree = pbm.binomial_tree(stock_price, number_of_model_steps, length_of_model_step, risk_free_rate, dividend_yield, stock_volatility)

Pour inspecter l'arbre, vous pouvez utiliser la méthode associée `print`. Les périodes de l'arbre sont indexées de $0$ à `number_of_model_steps`. Jetons un coup d'oeil au résultat au temps $1$.

Notez que les périodes à imprimer doivent être données sous forme de `list` : par exemple, `[0, 1, 2]`.

Dans le résultat ci-dessous, vous verrez que

- Les périodes sont numérotées et séparées par des en-têtes
- Pour chaque période, tous les états possibles du prix des actions sont imprimés.
- Tous les états sont nommés en suivant la convention qui dit combien de mouvements `U` et `D` sont nécessaires pour atteindre un état donné, par exemple `U1-D0` pour l'état "up" à la période $1$ et `U0-D1` pour l'état "down" à la période 1.

Il y a des informations supplémentaires dans le résultat :

- **"Up" Transition Probability**: $p^{\star}$ dans nos diapos
- **Probability of State** : la probabilité d'atteindre un état donné via toutes les trajectoires possibles
- **Multi-Period Discount Factor**: le facteur d'actualisation **de** la période dans l'en-tête **jusqu'à** la période finale ; $e^{-r_f (K-k) h}$ où la période finale est $K$ et la période actuelle est $k$.
- **Single-Period Discount Factor**: $e^{-r_f h}$
- Les facteurs "Up" et "Down" sont $u$ et $d$.

In [19]:
stock_price_tree.print([1])

--------------------------------------------------------------------
In Tree At Period 1:
--------------------------------------------------------------------
In State U1-D0
Stock price = 106.031747
Probability of State = 0.485570
"Up" transition probability (p-star) = 0.485570
Multi-Period Discount Factor = 0.996672
Single-Period Discount Factor = 0.998335
"Up" factor = 1.060317
"Down" factor = 0.944687


In State U0-D1
Stock price = 94.468693
Probability of State = 0.514430
"Up" transition probability (p-star) = 0.485570
Multi-Period Discount Factor = 0.996672
Single-Period Discount Factor = 0.998335
"Up" factor = 1.060317
"Down" factor = 0.944687




### Exercice:

Retournez à vos diapositives et utilisez les paramètres du prix des actions de la section précédente pour construire un arbre binomial forward à la main. Utilisez ensuite ce module pour vérifier vos résultats.

### Prix d'une option de vente européenne

Vous pouvez utiliser le modèle pour évaluer une option de vente européenne. L'idée est d'abord de construire l'arbre qui décrit l'évolution du prix de l'action, puis de tarifier l'option *sur cet arbre*.

Cela signifie que l'un des paramètres clés de l'option, le **temps à l'échéance**, dépend de l'arbre !

Tarifions l'option avec les paramètres suivants :

- Expiration dans six mois
- Prix d'exercice de \$95

#### Définir l'arbre des prix de l'action

Notre arbre binomial existant, `stock_price_tree`, décrit l'évolution du prix sur trois mois. Par conséquent, nous ne pouvons pas l'utiliser pour tarifier l'option, et nous devons créer un nouvel arbre.

Nous allons nous en tenir à un arbre à trois périodes. Cela signifie que nous avons besoin de périodes de deux mois, c'est-à-dire $h = 2/12$ sur l'arbre.

In [20]:
stock_price = 100
number_of_model_steps = 3 # three steps
length_of_model_step = 2/12 # a month is 1/12 years
risk_free_rate = 0.02
dividend_yield = 0.01
stock_volatility = 0.2 # annualized

stock_price_tree = pbm.binomial_tree(stock_price, number_of_model_steps, length_of_model_step, risk_free_rate, dividend_yield, stock_volatility)

#### Définir l'option

Pour définir une option de vente européenne, nous allons utiliser la classe `european_put` du module `pbm`. Voici le résultat du `help(pbm.european_put)`.

Un `european_put` est un `derivative`, donc si vous êtes intéressés, vous pouvez suivre en lisant `help(pbm.derivative)` et le code du module.

In [21]:
help(pbm.european_put)

Help on class european_put in module pedagogical_binomial_model:

class european_put(derivative)
 |  european_put(strike_price)
 |  
 |  European put
 |  
 |  Method resolution order:
 |      european_put
 |      derivative
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, strike_price)
 |      Define an European Put
 |      
 |      Parameters
 |      ----------
 |      strike_price : float
 |          strike price of the option
 |  
 |  ----------------------------------------------------------------------
 |  Methods inherited from derivative:
 |  
 |  print(self, step_indexes=None)
 |  
 |  set_tree(self, binomial_tree)
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors inherited from derivative:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)



La liste d'aide ci-dessus nous indique comment définir un `european_put` :
```
|  european_put(strike_price)
...
|  Methods defined here:
|  
|  __init__(self, strike_price)
|      Define an European Put
|      
|      Parameters
|      ----------
|      strike_price : float
|          strike price of the option
```

Il suffit de fixer un `strike_price` et d'écrire `mon_option = pbm.european_put(strike_price)`.

In [22]:
strike_price = 95
my_euro_put = pbm.european_put(strike_price)

#### Tarifier l'option

Après avoir défini l'arbre des prix des actions et l'option, nous pouvons tarifier l'option.

Pour ce faire, nous utilisons la méthode `pricing()` de l'arbre des prix des actions` sur l'option.

In [23]:
my_euro_put = stock_price_tree.pricing(my_euro_put)

Apparemment, rien ne s'est passé.

Cependant, `my_euro_put` contient maintenant un arbre binomial complet avec toutes les informations de prix incrémentales !

La classe `derivative` et toutes les classes liées (c'est-à-dire les calls et les puts européens et américains dans `pbm`) ont une méthode `print` qui est identique à la méthode `print` pour l'arbre binomial que nous avons examiné ci-dessus.

Pour connaître la valeur de l'option de vente au temps $0$, vous pouvez simplement taper `my_euro_put.print([0])` et voir dans le champ `"Derivative"` que l'option vaut $3.1966.

In [24]:
my_euro_put.print([0])

--------------------------------------------------------------------
In Tree At Period 0:
--------------------------------------------------------------------
In State 0
Stock price = 100.000000
Probability of State = 1.000000
"Up" transition probability (p-star) = 0.479599
Multi-Period Discount Factor = 0.990050
Single-Period Discount Factor = 0.996672
"Up" factor = 1.086886
"Down" factor = 0.923132
Derivative = 3.196583
Repl. Delta = -0.300772
Repl. Bond = 33.273770




#### Exercice : Tarifier l'option en utilisant le flux monétaire terminal espéré

Pour les produits dérivés européens, nous pouvons utiliser l'approche alternative à la tarification. Avec $K$ périodes de longueur $h$ années, le prix d'un dérivé $G(S_K)$ est :
$$
    P = e^{-r_f \times h \times K}E^{\star}\left[ G(S_K) \right]\, ,
$$
l'espérance risque-neutre du flux monétaire, actualisée au taux sans risque.

Pour voir les probabilités des états terminaux ainsi que les prix des actions dans ces états, tapez `my_euro_put.print([3])`.

In [25]:
my_euro_put.print([3])

--------------------------------------------------------------------
In Tree At Period 3:
--------------------------------------------------------------------
In State U3-D0
Stock price = 128.395990
Probability of State = 0.110315
"Up" transition probability (p-star) = 0.479599
Multi-Period Discount Factor = 1.000000
Single-Period Discount Factor = 0.996672
"Up" factor = 1.086886
"Down" factor = 0.923132
Derivative = 0.000000


In State U2-D1
Stock price = 109.051456
Probability of State = 0.359100
"Up" transition probability (p-star) = 0.479599
Multi-Period Discount Factor = 1.000000
Single-Period Discount Factor = 0.996672
"Up" factor = 1.086886
"Down" factor = 0.923132
Derivative = 0.000000


In State U1-D2
Stock price = 92.621429
Probability of State = 0.389651
"Up" transition probability (p-star) = 0.479599
Multi-Period Discount Factor = 1.000000
Single-Period Discount Factor = 0.996672
"Up" factor = 1.086886
"Down" factor = 0.923132
Derivative = 2.378571


In State U0-D3
Stock pr

Dans le champ `"Derivative"` vous pouvez voir les valeurs du payoff $G(S_K)$ qui ici est $\max(95 - S_K, 0)$. Utilisez ensuite la `"Probability of State"` (Probabilité d'état) et le `"Multi-Period Discount Factor"` (Facteur d'actualisation multi-périodes) approprié de la période $0$ à la période $K$ pour calculer le prix du Put Européen avec un stylo et du papier.

#### Extra-scolaire (pour les nerds)

Vous pouvez récupérer toutes les informations qui sont imprimées sur l'arbre pour les utiliser comme données dans votre cod. Les extraits de code suivants extraient les probabilités risque-neutres des états au dernier nœud de l'arbre, ainsi que les payoffs correspondants.

Au final, l'attribut `.trunk` de l'arbre des prix de l'option est un `dictionnaire`.

In [26]:
type(my_euro_put.pricing_tree.trunk)

dict

In [27]:
import numpy as np
rn_probs = [my_euro_put.pricing_tree.trunk["Period 3"][state]["Probability of State"] for state in my_euro_put.pricing_tree.trunk["Period 3"].keys()]
rn_probs = np.array(rn_probs)

print("Voici les probabilités risque-neutres, converties en un array numpy.")
print(rn_probs)

Voici les probabilités risque-neutres, converties en un array numpy.
[0.110315   0.35910036 0.38965103 0.14093361]


In [28]:
payoffs = [my_euro_put.pricing_tree.trunk["Period 3"][state]["Derivative"] for state in my_euro_put.pricing_tree.trunk["Period 3"].keys()]
payoffs = np.array(payoffs)

print("Voici les payoffs")
print(payoffs)

Voici les payoffs
[ 0.          0.          2.37857115 16.33319997]


Vous pouvez maintenant calculer l'espérance risque-neutre et l'actualiser à la période 0$ :

In [29]:
put_price = np.sum(rn_probs * payoffs)
put_price = my_euro_put.pricing_tree.trunk["Period 0"]["State 0"]["Multi-Period Discount Factor"] * put_price

print("Le prix de l'option de vente est ${0:1.4f}.".format(put_price))

Le prix de l'option de vente est $3.1966.


### Prix d'une option de vente américaine et comparaison avec l'option européenne

L'option américaine est plus *optionnelle* que l'option européenne, car elle peut être exercée à tout moment avant l'expiration. Si elle est plus optionnelle... alors sa valeur devrait être **supérieure** à celle de l'option européenne.

La différence entre les prix des options américaines et européennes, par ailleurs identiques, s'appelle la **prime d'exercice anticipé** (*early exercise premium*).

Nous pouvons utiliser notre modèle binomial pour évaluer la prime de l'option américaine.

Une option de vente américaine peut être instanciée à partir de la classe `american_put`. Nous utiliserons le même arbre et le même prix d'exercice que précédemment.

In [30]:
my_amer_put = pbm.american_put(95)
my_amer_put = stock_price_tree.pricing(my_amer_put)

In [31]:
my_amer_put.print([0])

--------------------------------------------------------------------
In Tree At Period 0:
--------------------------------------------------------------------
In State 0
Stock price = 100.000000
Probability of State = 1.000000
"Up" transition probability (p-star) = 0.479599
Multi-Period Discount Factor = 0.990050
Single-Period Discount Factor = 0.996672
"Up" factor = 1.086886
"Down" factor = 0.923132
Derivative = 3.243454
Exercise Derivative = False




In [32]:
my_euro_put.print([0])

--------------------------------------------------------------------
In Tree At Period 0:
--------------------------------------------------------------------
In State 0
Stock price = 100.000000
Probability of State = 1.000000
"Up" transition probability (p-star) = 0.479599
Multi-Period Discount Factor = 0.990050
Single-Period Discount Factor = 0.996672
"Up" factor = 1.086886
"Down" factor = 0.923132
Derivative = 3.196583
Repl. Delta = -0.300772
Repl. Bond = 33.273770




Le prix du put américain est de 3,2435 \$ et le prix du put européen est de 3,1966 \$.

La **prime d'exercice anticipé** est égale à 0,0469 \$ !

## Autres exercices

Jouez avec ce module. Utilisez-le pour répondre aux questions suivantes :

### Exercice 1

Quelle est la prime d'exercice anticipé pour les options d'achat si $\delta = 0 $ ?

### Exercice 2

Quelle est la prime d'exercice anticipé pour les options de vente si $r = 0$ ?

### Exercice 3

Que se passe-t-il avec le prix d'une option européenne si la grille de temps du modèle binomial devient *plus dense* ?

Nous entendons par là que nous pourrions garder l'horizon du modèle constant mais le décrire en un plus grand nombre d'étapes. Ci-dessus, nous avons utilisé trois étapes pour un horizon de six mois. Et si nous en utilisions six (c'est-à-dire des étapes mensuelles) ? Et si nous utilisions 180 (c'est-à-dire des étapes quotidiennes) ?

D'abord, définissez un arbre où la `length_of_model_step = 1/365` et le `number_of_model_steps = 180`.

Ensuite, écrivez une fonction qui évalue la formule d'évaluation des options de Black-Scholes et comparez les prix de la formule aux prix d'un modèle binomial avec de très nombreux pas (disons, 1 pas par jour).

Tarifiez une option européenne (call ou put) avec les deux méthodes.

### Exercice 4

Les prix des options européennes de vente et d'achat **avec le même prix d'exercice et la même échéance** sont fortement liés les uns aux autres ! Cette relation est appelée la **parité put-call** (*put-call parity*). Elle peut être énoncée comme suit :
$$
    C - P = S e^{-\delta T} - K \times e^{-r_f T}\, ,
$$
où $C$ et $P$ sont, respectivement, les prix des options d'achat et de vente, $S$ est le prix de l'action, $K$ est le prix d'exercice des deux options, $r_f$ est le taux sans risque, $\delta$ est le rendement du dividende et $T$ est la maturité des deux options.

Utilisez le modèle binomial pour :

1. Démontrer que la relation de parité put-call existe bel et bien,
2. Examiner comment l'écart par rapport à la relation dépend de $\delta$ pour les options américaines.
